<a href="https://colab.research.google.com/github/SH22Hwang/KoGES_OP/blob/master/OP_20230515.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 골다공증

다양한 모델로 돌려보았다~

- Traditional ML
  + Linear Model
  + KNN
  + Decison Tree
  + SVM
- GBDT
  + XGBoost
  + LGBM
- DNN
  + MLP
  + SAINT

## 개발환경

- PyTorch

## 문제점

- classification 모델인데, 어느 한 쪽으로만 예측하는 경우 발생
  - 알고보니, 예측값을 int64로 취급한 것 같음.
  - classification이 아니었다.

- Undersampling 후 결과 비교



In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as fn
from torch.utils.data import DataLoader, TensorDataset, Dataset

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from sklearn.metrics import mean_squared_error, roc_auc_score # roc_auc_score(y_test, y_pred)

from xgboost import XGBClassifier
import lightgbm as lgb

import numpy as np
import pandas as pd

from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from imblearn.under_sampling import RandomUnderSampler
from imblearn.over_sampling import SMOTE

In [ ]:
# Statistical Accuracy Measurement Comprehensive Gift Set
def SAMCGS(y_test, y_pred):
  # 정확도 종합 선물 세트
  accuracy = accuracy_score(y_test, y_pred)
  conf_mat = confusion_matrix(y_test, y_pred)
  # tn, fp, fn, tp = conf_mat.ravel()

  precision = precision_score(y_test, y_pred, zero_division=1) # tp / (tp + fp)
  recall = recall_score(y_test, y_pred, zero_division=1)       # tp / (tp + fn)
  f1 = f1_score(y_test, y_pred, zero_division=1) # 2 * (precision * recall) / (precision + recall)

  print(f'''
  Confusion Matrix:\n{conf_mat}\n
  Accuracy: {accuracy:.3f}
  Precison: {precision:.3f}
  Recall  : {recall:.3f}
  F1 Score: {f1:.3f}'''
  )

  # return precision, recall, f1

In [ ]:
# 데이터 불러오기
# _premeno: 폐경 전 여성
# _postmeno: 폐경 후 여성
X_premeno = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PreProcessed/X_premeno_20230614.csv', index_col=0)
X_postmeno = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PreProcessed/X_postmeno_20230614.csv', index_col=0)
y_premeno = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PreProcessed/y_premeno_20230614.csv', index_col=0)
y_postmeno = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/PreProcessed/y_postmeno_20230614.csv', index_col=0)

In [ ]:
# UnderSampling, OverSampling
# rus = RandomUnderSampler(random_state=42)
smt = SMOTE(random_state=42)

# X_pre, y_pre = rus.fit_resample(X_premeno, y_premeno)
# X_post, y_post = rus.fit_resample(X_postmeno, y_postmeno)
X_pre, y_pre = smt.fit_resample(X_premeno, y_premeno)
X_post, y_post = smt.fit_resample(X_postmeno, y_postmeno)

In [ ]:
X_pre_tensor = torch.tensor(X_pre.values).float()
y_pre_tensor = torch.tensor(y_pre.values).float()
X_post_tensor = torch.tensor(X_post.values).float()
y_post_tensor = torch.tensor(y_post.values).float()

In [ ]:
print(y_postmeno.value_counts())
print(y_post.value_counts())

OP
0     2109
1      825
dtype: int64
OP
0     2109
1     2109
dtype: int64


## train test split

- train set: 0.8
  - train set: 0.64
  - val set: 0.16
- test set: 0.2

[참고 자료](https://ysyblog.tistory.com/69)

In [ ]:
# train / test
X_pre_train, X_pre_test, y_pre_train, y_pre_test \
= train_test_split(X_pre, y_pre, test_size=0.2, stratify=y_pre, random_state=415) # stratify: target의 class비율에 맞춰서 분리

X_post_train, X_post_test, y_post_train, y_post_test \
= train_test_split(X_post, y_post, test_size=0.2, stratify=y_post, random_state=415)

# train / val
X_pre_train, X_pre_val, y_pre_train, y_pre_val \
= train_test_split(X_pre_train, y_pre_train, test_size=0.2, stratify=y_pre_train, random_state=415)

X_post_train, X_post_val, y_post_train, y_post_val \
= train_test_split(X_post_train, y_post_train, test_size=0.2, stratify=y_post_train, random_state=415)

# tensor / train / test
X_pre_train_tensor, X_pre_test_tensor, y_pre_train_tensor, y_pre_test_tensor \
= train_test_split(X_pre_tensor, y_pre_tensor, test_size=0.2, stratify=y_pre, random_state=415)

X_post_train_tensor, X_post_test_tensor, y_post_train_tensor, y_post_test_tensor \
= train_test_split(X_post_tensor, y_post_tensor, test_size=0.2, stratify=y_post, random_state=415)

# tensor / train / val
X_pre_train_tensor, X_pre_val_tensor, y_pre_train_tensor, y_pre_val_tensor \
= train_test_split(X_pre_train_tensor, y_pre_train_tensor, test_size=0.2, stratify=y_pre_train_tensor, random_state=415)

X_post_train_tensor, X_post_val_tensor, y_post_train_tensor, y_post_val_tensor \
= train_test_split(X_post_train_tensor, y_post_train_tensor, test_size=0.2, stratify=y_post_train_tensor, random_state=415)

## Decision Tree

In [ ]:
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
from sklearn.tree import export_graphviz
import graphviz

In [ ]:
# Defining the hyper parameters
hps = {
    'max_depth': 5,
    'min_samples_split': 4
}

# Loading the tree object
tree_pre = DecisionTreeClassifier(**hps)
tree_post = DecisionTreeClassifier(**hps)

tree_pre.fit(X_pre, y_pre)
tree_post.fit(X_post, y_post)

DecisionTreeClassifier(max_depth=5, min_samples_split=4)

In [ ]:
y_pre_pred = tree_pre.predict(X_pre)
y_post_pred = tree_post.predict(X_post)

In [ ]:
print("폐경 전 여성, Decision Tree")
SAMCGS(y_pre, y_pre_pred)
print("폐경 전 여성, Decision Tree")
SAMCGS(y_post, y_post_pred)

폐경 전 여성, Decision Tree

  Confusion Matrix:
[[1346  305]
 [  99 1552]]

  Accuracy: 0.878
  Precison: 0.836
  Recall  : 0.940
  F1 Score: 0.885
폐경 전 여성, Decision Tree

  Confusion Matrix:
[[1558  551]
 [ 584 1525]]

  Accuracy: 0.731
  Precison: 0.735
  Recall  : 0.723
  F1 Score: 0.729


### 그래프

In [ ]:
# export_graphviz(tree, out_file="tree.dot",
#                 class_names='FT',
#                 feature_names=X_post.columns,
#                 impurity=True, filled=True)

In [ ]:
# with open("tree.dot") as f:
#     dot_graph = f.read()

# graphviz.Source(dot_graph, filename='tree.png',format='png')

In [ ]:
# !dot -Tpng tree.dot -o tree.png

# tree_img = plt.imread('./tree.png')

# plt.figure(num=None, figsize=(32, 9), dpi=80, facecolor='w', edgecolor='k')
# plt.imshow(tree_img)

### importance

In [ ]:
# # Extracting the importances by sklearn
# importances_sk = tree.feature_importances_

# # Creating a dataframe with the feature importance by sklearn
# feature_importance_sk = {}
# for i, feature in enumerate(X_post.columns.tolist()):
#     feature_importance_sk[feature] = round(importances_sk[i], 3)

# print(f"Feature importance by sklearn: {feature_importance_sk}")

In [ ]:
# importances = tree.feature_importances_

# tree_importances = pd.Series(importances, index=X_post.columns).sort_values()
# fig, ax = plt.subplots(figsize=(10, 40))
# tree_importances.plot.barh(ax=ax)
# ax.set_title("OP feature importance with all variable")
# ax.set_ylabel("feature importance")

In [ ]:
# tree_importances

## KNN

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

knn_premeno = KNeighborsClassifier(n_neighbors=2)
knn_postmeno = KNeighborsClassifier(n_neighbors=2)

knn_premeno.fit(X_pre_train, y_pre_train)
knn_postmeno.fit(X_post_train, y_post_train)

y_premeno_pred = knn_premeno.predict(X_pre_test)
y_postmeno_pred = knn_postmeno.predict(X_post_test)

/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)
/usr/local/lib/python3.10/dist-packages/sklearn/neighbors/_classification.py:215: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


In [ ]:
print("폐경 전 여성, KNN")
SAMCGS(y_pre_test, y_premeno_pred)
print("폐경 후 여성, KNN")
SAMCGS(y_post_test, y_postmeno_pred)

폐경 전 여성, KNN

  Confusion Matrix:
[[231  99]
 [  0 331]]

  Accuracy: 0.850
  Precison: 0.770
  Recall  : 1.000
  F1 Score: 0.870
폐경 후 여성, KNN

  Confusion Matrix:
[[194 228]
 [ 33 389]]

  Accuracy: 0.691
  Precison: 0.630
  Recall  : 0.922
  F1 Score: 0.749


## SVM

In [ ]:
# Define the linear SVM model
class SVM(nn.Module):
    def __init__(self, input_dim):
        super(SVM, self).__init__()
        self.fc1 = nn.Linear(input_dim, 1)

    def forward(self, x):
        x = self.fc1(x)
        return x

In [ ]:
svm_model_pre = SVM(X_pre_train.shape[1])
svm_model_post = SVM(X_post_train.shape[1])

# Define the loss function and optimizer
criterion_pre = nn.HingeEmbeddingLoss()
criterion_post = nn.HingeEmbeddingLoss()

optimizer_pre = optim.SGD(svm_model_pre.parameters(), lr=0.01)
optimizer_post = optim.SGD(svm_model_post.parameters(), lr=0.01)

In [ ]:
# Train the model
for epoch in range(100):
    optimizer_pre.zero_grad()
    optimizer_post.zero_grad()

    output_pre = svm_model_pre(X_pre_train_tensor).squeeze()
    output_post = svm_model_post(X_post_train_tensor).squeeze()

    loss_pre = criterion_pre(output_pre, 2 * y_pre_train_tensor.float()-1)
    loss_post = criterion_pre(output_post, 2 * y_post_train_tensor.float()-1)

    loss_pre.backward()
    loss_post.backward()

    optimizer_pre.step()
    optimizer_post.step()

# Make predictions on the test set
y_pre_pred = torch.sign(svm_model_pre(X_pre_test_tensor).squeeze()).detach().numpy()
y_post_pred = torch.sign(svm_model_post(X_post_test_tensor).squeeze()).detach().numpy()

In [ ]:
# y_premeno_pred = np.where(y_premeno_pred < 0, 0, 1)
# unique, counts = np.unique(y_premeno_pred, return_counts=True)
# print(dict(zip(unique, counts)))

In [ ]:
y_post_test.value_counts()

OP
0     422
1     422
dtype: int64

In [ ]:
# y_postmeno_pred = np.where(y_postmeno_pred < 0, 0, 1)
unique, counts = np.unique(y_premeno_pred, return_counts=True)
print(dict(zip(unique, counts)))

{0: 231, 1: 430}


In [ ]:
print("폐경 전 여성, SVM")
SAMCGS(y_pre_test.to_numpy(), y_premeno_pred)
print("폐경 후 여성, SVM")
SAMCGS(y_post_test, y_postmeno_pred)

폐경 전 여성, SVM

  Confusion Matrix:
[[231  99]
 [  0 331]]

  Accuracy: 0.850
  Precison: 0.770
  Recall  : 1.000
  F1 Score: 0.870
폐경 후 여성, SVM

  Confusion Matrix:
[[  0 422]
 [  0 422]]

  Accuracy: 0.500
  Precison: 0.500
  Recall  : 1.000
  F1 Score: 0.667


## XGBoost

In [ ]:
from sklearn.model_selection import GridSearchCV

# create model instance
xgb_pre = XGBClassifier(n_estimators=300, # 나무의 개수
                        # early_stopping_rounds = 100,
                        max_depth=5,
                        learning_rate=0.01,
                        objective='binary:logistic',
                        silent=1)

xgb_post = XGBClassifier(n_estimators=300, # 나무의 개수
                        #  early_stopping_rounds = 100,
                         max_depth=5,
                         learning_rate=0.01,
                         objective='binary:logistic',
                         silent=1)

# 후보 파라미터 선정
params = {'max_depth':[5,7], 'min_child_weight':[1,3], 'colsample_bytree':[0.5,0.75]}

# gridsearchcv 객체 정보 입력(어떤 모델, 파라미터 후보, 교차검증 몇 번)
gridcv_pre = GridSearchCV(xgb_pre, param_grid=params, cv=3)
gridcv_post = GridSearchCV(xgb_post, param_grid=params, cv=3)

gridcv_pre.fit(X_pre_train, y_pre_train,
               early_stopping_rounds=30,
               eval_metric='auc',
               eval_set=[(X_pre_val, y_pre_val)])

gridcv_post.fit(X_post_train, y_post_train,
                early_stopping_rounds=30,
                eval_metric='auc',
                eval_set=[(X_post_val, y_post_val)])

In [ ]:
print(gridcv_pre.best_params_)
print(gridcv_post.best_params_)

{'colsample_bytree': 0.5, 'max_depth': 7, 'min_child_weight': 1}
{'colsample_bytree': 0.5, 'max_depth': 7, 'min_child_weight': 1}


In [ ]:
# create model instance
xgb_premeno = XGBClassifier(n_estimators=1000, # 나무의 개수
                            early_stopping_rounds = 100,
                            learning_rate=0.01,
                            objective='binary:logistic',
                            silent=1,
                            **gridcv_pre.best_params_)

xgb_postmeno = XGBClassifier(n_estimators=1000, # 나무의 개수
                            early_stopping_rounds = 100,
                            learning_rate=0.01,
                            objective='binary:logistic',
                            silent=1,
                            **gridcv_post.best_params_)

In [ ]:
# fit model
xgb_premeno.fit(X_pre_train, y_pre_train,
                eval_metric='auc',
                eval_set=[(X_post_val, y_post_val)])

xgb_postmeno.fit(X_post_train, y_post_train,
                 eval_metric='auc',
                 eval_set=[(X_post_val, y_post_val)])

# make predictions
y_premeno_pred = xgb_premeno.predict(X_pre_test)
y_postmeno_pred = xgb_postmeno.predict(X_post_test)

[09:01:43] WARNING: ../src/learner.cc:767: 
Parameters: { "silent" } are not used.

[0]	validation_0-auc:0.57274
[1]	validation_0-auc:0.59175
[2]	validation_0-auc:0.59241
[3]	validation_0-auc:0.59201
[4]	validation_0-auc:0.58998


/usr/local/lib/python3.10/dist-packages/xgboost/sklearn.py:835: UserWarning: `eval_metric` in `fit` method is deprecated for better compatibility with scikit-learn, use `eval_metric` in constructor or`set_params` instead.
  warnings.warn(


[5]	validation_0-auc:0.58294
[6]	validation_0-auc:0.58840
[7]	validation_0-auc:0.58697
[8]	validation_0-auc:0.58828
[9]	validation_0-auc:0.58865
[10]	validation_0-auc:0.57916
[11]	validation_0-auc:0.58997
[12]	validation_0-auc:0.59919
[13]	validation_0-auc:0.60305
[14]	validation_0-auc:0.60913
[15]	validation_0-auc:0.61769
[16]	validation_0-auc:0.62020
[17]	validation_0-auc:0.61399
[18]	validation_0-auc:0.61591
[19]	validation_0-auc:0.61769
[20]	validation_0-auc:0.62049
[21]	validation_0-auc:0.61707
[22]	validation_0-auc:0.61637
[23]	validation_0-auc:0.62216
[24]	validation_0-auc:0.62350
[25]	validation_0-auc:0.62421
[26]	validation_0-auc:0.62521
[27]	validation_0-auc:0.62654
[28]	validation_0-auc:0.62543
[29]	validation_0-auc:0.62833
[30]	validation_0-auc:0.62716
[31]	validation_0-auc:0.62777
[32]	validation_0-auc:0.62996
[33]	validation_0-auc:0.63059
[34]	validation_0-auc:0.63032
[35]	validation_0-auc:0.63232
[36]	validation_0-auc:0.63216
[37]	validation_0-auc:0.63332
[38]	validation

In [ ]:
print("폐경 전 여성, XGBoost")
SAMCGS(y_pre_test, y_premeno_pred)
print("폐경 후 여성, XGBoost")
SAMCGS(y_post_test, y_postmeno_pred)

폐경 전 여성, XGBoost

  Confusion Matrix:
[[328   2]
 [ 11 320]]

  Accuracy: 0.980
  Precison: 0.994
  Recall  : 0.967
  F1 Score: 0.980
폐경 후 여성, XGBoost

  Confusion Matrix:
[[346  76]
 [100 322]]

  Accuracy: 0.791
  Precison: 0.809
  Recall  : 0.763
  F1 Score: 0.785


## LGBM

In [ ]:
# # Define the LightGBM model
# params = {
#     'objective': 'binary',
#     'metric': 'auc',
#     'boosting_type': 'gbdt',
#     'num_leaves': 31,
#     'learning_rate': 0.05,
#     'feature_fraction': 0.9,
#     'bagging_fraction': 0.8,
#     'bagging_freq': 5,
#     'verbose': 0
# }

# train_data = lgb.Dataset(X_train, label=y_train)
# test_data = lgb.Dataset(X_test, label=y_test)

# # Train the model
# model = lgb.train(params, train_data, num_boost_round=100, valid_sets=[train_data, test_data], early_stopping_rounds=10)

# # Make predictions on test set
# y_pred = model.predict(X_test)

clf_premeno = lgb.LGBMClassifier()
clf_premeno.fit(X_pre_train, y_pre_train) # LightGBMError: Do not support special JSON characters in feature name.
                                                  #
y_pre_pred=clf_premeno.predict(X_pre_test)

clf_postmeno = lgb.LGBMClassifier()
clf_postmeno.fit(X_post_train, y_post_train)
y_post_pred=clf_postmeno.predict(X_post_test)

/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:99: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.10/dist-packages/sklearn/preprocessing/_label.py:134: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the 

In [ ]:
print("폐경 전 여성, LGBM")
SAMCGS(y_pre_test, y_pre_pred)
print("폐경 후 여성, LGBM")
SAMCGS(y_post_test, y_post_pred)

폐경 전 여성, LGBM

  Confusion Matrix:
[[324   6]
 [  8 323]]

  Accuracy: 0.979
  Precison: 0.982
  Recall  : 0.976
  F1 Score: 0.979
폐경 후 여성, LGBM

  Confusion Matrix:
[[353  69]
 [102 320]]

  Accuracy: 0.797
  Precison: 0.823
  Recall  : 0.758
  F1 Score: 0.789


## MLP

- 이전에 가장 좋았던 코드 사용
  - 고혈압 분석 모델 참고
  - hidden:
    - layer: 5
    - dimension: 32
  - output:
    - dimmension: 1
    - activation: Sigmoid

- 내가 할 코드
  - hidden:
    - layer:
    - dimension:
  - output:
    - dimmension:
    - activation: Sigmoid


### 블로그에서 가져온 클래스
출처: https://data-science-hi.tistory.com/188


- X_premeno_train
- X_premeno_test
- y_premeno_train
- y_premeno_test

- X_postmeno_train
- X_postmeno_test
- y_postmeno_train
- y_postmeno_test

In [ ]:
# torch의 Dataset 을 상속.
class TensorData(Dataset):

    def __init__(self, x_data, y_data):
        self.x_data = torch.FloatTensor(x_data)
        self.y_data = torch.FloatTensor(y_data)
        self.len = self.y_data.shape[0]

    def __getitem__(self, index):
        return self.x_data[index], self.y_data[index]

    def __len__(self):
        return self.len

In [ ]:
# 학습 데이터, 시험 데이터 배치 형태로 구축하기
# premeno, postmeno 따로 생성

premeno_train_sets = TensorData(X_pre_train_tensor, y_pre_train_tensor)
premeno_train_loader = torch.utils.data.DataLoader(premeno_train_sets, batch_size=32, shuffle=True)

premeno_test_sets = TensorData(X_pre_test_tensor, y_pre_test_tensor)
premeno_test_loader = torch.utils.data.DataLoader(premeno_test_sets, batch_size=32, shuffle=False)

postmeno_train_sets = TensorData(X_post_train_tensor, y_post_train_tensor)
postmeno_train_loader = torch.utils.data.DataLoader(postmeno_train_sets, batch_size=32, shuffle=True)

postmeno_test_sets = TensorData(X_post_test_tensor, y_post_test_tensor)
postmeno_test_loader = torch.utils.data.DataLoader(postmeno_test_sets, batch_size=32, shuffle=False)

In [ ]:
# input, hidden dim 매개변수로 따로 넣어주기

class Regressor(nn.Module):
    def __init__(self, input_dim, hidden_dim):
        super().__init__() # 모델 연산 정의
        self.fc1 = nn.Linear(input_dim, hidden_dim, bias=True) # 입력층(13) -> 은닉층1(50)으로 가는 연산
        self.fc2 = nn.Linear(hidden_dim, hidden_dim, bias=True) # 은닉층1(50) -> 은닉층2(30)으로 가는 연산
        self.fc3 = nn.Linear(hidden_dim, 1, bias=True) # 은닉층2(30) -> 출력층(1)으로 가는 연산
        self.dropout = nn.Dropout(0.2) # 연산이 될 때마다 20%의 비율로 랜덤하게 노드를 없앤다.

    def forward(self, x): # 모델 연산의 순서를 정의
        x = fn.relu(self.fc1(x)) # Linear 계산 후 활성화 함수 ReLU를 적용한다.
        x = self.dropout(fn.relu(self.fc2(x))) # 은닉층2에서 드랍아웃을 적용한다.(즉, 30개의 20%인 6개의 노드가 계산에서 제외된다.)
        x = fn.relu(self.fc3(x)) # Linear 계산 후 활성화 함수 ReLU를 적용한다.

        return x

# 주의 사항
# 드랍아웃은 과적합(overfitting)을 방지하기 위해 노드의 일부를 배제하고 계산하는 방식이기 때문에 절대로 출력층에 사용해서는 안 된다.

In [ ]:
mlp_premeno = Regressor(X_pre_train.shape[1], 32)
mlp_postmeno = Regressor(X_post_train.shape[1], 32)

In [ ]:
# criterion = nn.MSELoss() critation은 earlystopping을 사용할 때 쓰는 것.
objective = nn.MSELoss()
optimizer = optim.Adam(mlp_postmeno.parameters(), lr=0.001, weight_decay=1e-7)

In [ ]:
loss_pre = [] # loss를 저장할 리스트.
loss_post = []
# n = len(trainloader)
len_pre_loader = len(premeno_train_loader)
len_post_loader = len(postmeno_train_loader)

for epoch in range(400):
  running_loss = 0.0 # 한 에폭이 돌 때 그안에서 배치마다 loss가 나온다. 즉 한번 학습할 때 그렇게 쪼개지면서 loss가 다 나오니 MSE를 구하기 위해서 사용한다.

  for i, data in enumerate(postmeno_train_loader, 0): # 무작위로 섞인 32개의 데이터가 담긴 배치가 하나씩 들어온다.
    inputs, values = data # data에는 X, Y가 들어있다.
    optimizer.zero_grad() # 최적화 초기화.
    outputs = mlp_premeno(inputs) # 모델에 입력값을 넣어 예측값을 산출한다.

    loss = objective(outputs, values) # 손실함수를 계산. error 계산.
    loss.backward() # 손실 함수를 기준으로 역전파를 설정한다.

    optimizer.step() # 역전파를 진행하고 가중치를 업데이트한다.

    running_loss += loss.item() # epoch 마다 평균 loss를 계산하기 위해 배치 loss를 더한다.

  loss_post.append(running_loss/len_post_loader) # MSE(Mean Squared Error) 계산

In [ ]:
# model: mlp_premeno dataloader: premeno_train_loader
def evaluation(model, dataloader):

  predictions = torch.tensor([], dtype=torch.float) # 예측값을 저장하는 텐서.
  actual = torch.tensor([], dtype=torch.float) # 실제값을 저장하는 텐서.

  with torch.no_grad():
    model.eval() # 평가를 할 땐 반드시 eval()을 사용해야 한다.

    for data in dataloader:
      inputs, values = data
      outputs = model(inputs)

      predictions = torch.cat((predictions, outputs), 0) # cat함수를 통해 예측값을 누적.
      actual = torch.cat((actual, values), 0) # cat함수를 통해 실제값을 누적.

  predictions = predictions.numpy() # 넘파이 배열로 변경.
  actual = actual.numpy() # 넘파이 배열로 변경.

  SAMCGS(actual.ravel().astype(np.int64), predictions.ravel().astype(np.int64))

  rmse = np.sqrt(mean_squared_error(predictions, actual)) # sklearn을 이용해 RMSE를 계산.

  return rmse

In [ ]:
# train_rmse = evaluation(mlp_premeno, premeno_train_loader) # 원래는 이렇게 하면 안되지만, 비교를 위해서 train을 넣어서 본다.
# test_rmse = evaluation(mlp_premeno, premeno_test_loader)
test_rmse = evaluation(mlp_postmeno, postmeno_test_loader)

# print(f'train rmse:{train_rmse}')
print(f'test rmse:{test_rmse}')


  Confusion Matrix:
[[422   0]
 [422   0]]

  Accuracy: 0.500
  Precison: 1.000
  Recall  : 0.000
  F1 Score: 0.000
test rmse:0.7071067690849304
